# Travelling Salesperson Problem solved using genetic algorithms

In [48]:
# Imports 
import numpy as np
import random
import pandas as pd
from geopy import distance,geocoders,Nominatim # to calculate distance on the surface
import folium
from datetime import datetime

In [49]:
# Parameters
n_cities = 15

n_population = 50

mutation_rate = 0.3

df = pd.read_csv('output.csv')

In [50]:
# Generating a list of coordenades representing each city
geolocator = geocoders.Nominatim(user_agent="my_email@myserver.com")
coordinates_list=[]
names_list = np.array(df['Street'].head(n_cities))
for city in names_list:
    location = geolocator.geocode(city)
    coordinates_list.append(list((location.latitude, location.longitude)))
    
cities_dict = { x:y for x,y in zip(names_list,coordinates_list)}

# Function to compute the distance between two points
def compute_city_distance_coordinates(a,b):
    return distance.distance((a[0], a[1]), (b[0],b[1])).km

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

cities_dict

{'24 Rue Tiquetonne': [48.8646943, 2.3485383],
 '4 Boulevard de Grenelle': [48.8540376, 2.2888741],
 '5 Rue Audran': [48.8851871, 2.3358188],
 '85 Boulevard de Sebastopol': [48.8648771, 2.3515534],
 '2 Rue du Faubourg Saint-Honore': [48.8685293, 2.3230081],
 '31 Avenue Secretan': [48.8813911, 2.3732117],
 '124 Rue du Faubourg Saint-Honore': [48.8721214, 2.3132623],
 "30 Avenue d'Italie": [48.830242850000005, 2.355210601316469],
 '43 Avenue de Clichy': [48.8869589, 2.3256654],
 '38 Rue Balard': [48.8428844, 2.2773907],
 '15 Rue Cail': [48.8826982, 2.360298],
 '79 Rue Lamarck': [48.8902312, 2.3362593],
 '14 Rue Papillon': [48.876317, 2.3471531],
 '138 Boulevard de la Villette': [48.8788385, 2.3705586],
 '55 Rue des Vinaigriers': [48.8731803, 2.3597091]}

## 1. Create the first population set
We randomly shuffle the cities N times where N=population_size

In [51]:
# First step: Create the first population set
def genesis(city_list, n_population):

    population_set = []
    for i in range(n_population):
        #Randomly generating a new solution
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
# population_set

## 2. Evaluate solutions fitness
The solutions are defined so that the first element on the list is the first city to visit, then the second, etc. and the last city is linked to the first.
The fitness function needs to compute the distance between subsequent cities.

In [52]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total

In [53]:
def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([36.11372239, 44.50310372, 41.00454377, 45.91905567, 41.91509519,
       55.53559009, 49.36737738, 48.15736603, 40.25252874, 48.16156143,
       44.09319485, 48.27328643, 52.54841799, 42.75144544, 53.21917539,
       44.67091187, 51.65419162, 47.57268014, 45.59581285, 45.70315969,
       42.95533695, 49.92403786, 50.44462368, 53.22999004, 46.62731551,
       37.79032247, 51.31082564, 42.90949672, 52.80546389, 53.17104846,
       44.86813091, 47.96207944, 45.16757665, 42.89772768, 46.6505741 ,
       47.19266766, 40.59702494, 52.22453166, 46.63156196, 48.41765155,
       42.60294194, 52.72852296, 46.53536865, 38.85976533, 53.64642205,
       47.27422998, 45.56557062, 47.67899059, 47.0499439 , 42.96315317])

# 3. Progenitors selection
I will select a new set of progenitors using the Roulette Wheel Selection. Generates a list of progenitor pairs where N= len(population_set) but at each position there are two solutions to merge

In [54]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit
    
    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    
    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]
    
    
    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
progenitor_list[0][2]

array(['31 Avenue Secretan', '4 Boulevard de Grenelle', '14 Rue Papillon',
       '85 Boulevard de Sebastopol', '2 Rue du Faubourg Saint-Honore',
       '24 Rue Tiquetonne', "30 Avenue d'Italie",
       '138 Boulevard de la Villette', '124 Rue du Faubourg Saint-Honore',
       '79 Rue Lamarck', '5 Rue Audran', '38 Rue Balard',
       '43 Avenue de Clichy', '55 Rue des Vinaigriers', '15 Rue Cail'],
      dtype=object)

# 4. Mating
For each pair of  parents we'll generate an offspring pair. Since we cannot repeat cities what we'll do is copy a random chunk from one progenitor and fill the blanks with the other progenitor.

In [55]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring
            
    
    
def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)
        
    return new_population_set

new_population_set = mate_population(progenitor_list)
new_population_set[0]

array(['24 Rue Tiquetonne', '2 Rue du Faubourg Saint-Honore',
       '38 Rue Balard', '15 Rue Cail', "30 Avenue d'Italie",
       '124 Rue du Faubourg Saint-Honore', '43 Avenue de Clichy',
       '79 Rue Lamarck', '4 Boulevard de Grenelle',
       '85 Boulevard de Sebastopol', '138 Boulevard de la Villette',
       '55 Rue des Vinaigriers', '5 Rue Audran', '14 Rue Papillon',
       '31 Avenue Secretan'], dtype=object)

# 5. Mutation
Now for each element of the new population we add a random chance of swapping

In [56]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)

        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring
    
    
def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)
mutated_pop[0]

array(['24 Rue Tiquetonne', '138 Boulevard de la Villette',
       '38 Rue Balard', '14 Rue Papillon', "30 Avenue d'Italie",
       '2 Rue du Faubourg Saint-Honore', '43 Avenue de Clichy',
       '79 Rue Lamarck', '4 Boulevard de Grenelle',
       '85 Boulevard de Sebastopol', '124 Rue du Faubourg Saint-Honore',
       '55 Rue des Vinaigriers', '5 Rue Audran', '15 Rue Cail',
       '31 Avenue Secretan'], dtype=object)

# 6. Stopping
To select the stopping criteria we'll need to create a loop to stop first. Then I'll set it to loop at 1000 iterations.

In [57]:
best_solution = [-1,np.inf,np.array([])]
for i in range(1000):
    if i%100==0: print(i, fitnes_list.min(), fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)
    
    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]
    
    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list)
    
    mutated_pop = mutate_population(new_population_set)

0 36.113722385618466 46.713902352517636 14/01/23 01:17
100 37.9766875100764 47.89663690767136 14/01/23 01:18
200 34.577128398588265 47.40704206130121 14/01/23 01:18
300 31.586464568501984 47.123947498580904 14/01/23 01:19
400 31.032961896181284 47.72987136149919 14/01/23 01:19
500 37.350786662783094 48.305675034094776 14/01/23 01:20
600 36.908884336149754 47.413981563791396 14/01/23 01:20
700 32.86148944768271 47.587926514691226 14/01/23 01:21
800 40.71064429394387 47.5078252923556 14/01/23 01:22
900 38.41505506377549 47.25323007254652 14/01/23 01:22


In [58]:
best_solution

[34,
 27.127028036007285,
 array([["30 Avenue d'Italie", '43 Avenue de Clichy', '79 Rue Lamarck',
         '5 Rue Audran', '15 Rue Cail', '31 Avenue Secretan',
         '138 Boulevard de la Villette', '2 Rue du Faubourg Saint-Honore',
         '55 Rue des Vinaigriers', '24 Rue Tiquetonne',
         '85 Boulevard de Sebastopol', '14 Rue Papillon',
         '124 Rue du Faubourg Saint-Honore', '4 Boulevard de Grenelle',
         '38 Rue Balard']], dtype=object)]

In [59]:
best_road = best_solution[2][0]
best_road

array(["30 Avenue d'Italie", '43 Avenue de Clichy', '79 Rue Lamarck',
       '5 Rue Audran', '15 Rue Cail', '31 Avenue Secretan',
       '138 Boulevard de la Villette', '2 Rue du Faubourg Saint-Honore',
       '55 Rue des Vinaigriers', '24 Rue Tiquetonne',
       '85 Boulevard de Sebastopol', '14 Rue Papillon',
       '124 Rue du Faubourg Saint-Honore', '4 Boulevard de Grenelle',
       '38 Rue Balard'], dtype=object)

In [60]:
cities_dict[best_road[0]]

[48.830242850000005, 2.355210601316469]

In [61]:
m = folium.Map(location=cities_dict[best_road[0]], zoom_start=12, tiles="Stamen Terrain")
loc = []
for el in best_road:
  loc.append(cities_dict[el])
  folium.Marker(
      cities_dict[el], popup=f"<i>{el}</i>"
  ).add_to(m)
loc.append(cities_dict[best_road[0]])
folium.PolyLine(loc,
                color='blue',
                weight=4,
                opacity=0.9).add_to(m)
m
#m.save("heat_map.html")